In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

from mlxtend.frequent_patterns import apriori, association_rules

import warnings
warnings.filterwarnings("ignore")

In [2]:
base = pd.read_excel('Base_case_analista_III.xlsx', header=1)

In [3]:
base.columns = ['Ciclo_Ciclo', 'Revendedor', 'Franqueado', 'Loja', 'Produto_Categoria',
       'Produto_Codigo', 'Produto_Descricao', 'Produto_Linha',
       'Pedido_Valor_Volume', 'Receita_Bruta', 'Receita_Liquida',
       'z_Numero_de_Pedidos']


In [4]:
base['Desconto'] = base['Receita_Bruta'] - base['Receita_Liquida']
base['Ciclo'] = base.apply(lambda x: 'Mar-2020' if x['Ciclo_Ciclo'] == 2020.03 else 'Fev-2020', axis=1)

In [5]:
base.dtypes

Ciclo_Ciclo            float64
Revendedor               int64
Franqueado               int64
Loja                     int64
Produto_Categoria       object
Produto_Codigo           int64
Produto_Descricao       object
Produto_Linha           object
Pedido_Valor_Volume      int64
Receita_Bruta          float64
Receita_Liquida        float64
z_Numero_de_Pedidos      int64
Desconto               float64
Ciclo                   object
dtype: object

In [6]:
base['Produto_Categoria'].unique()

array(['PERFUMARIA MASCULINA', 'ACESSÓRIOS', 'SUPORTE À VENDA',
       'PERFUMARIA FEMININA', 'GIFTS', 'CATALOGO VENDA DIRETA'],
      dtype=object)

In [7]:
len(base['Loja'].unique())

892

In [8]:
len(base['Franqueado'].unique())

504

In [9]:
len(base['Revendedor'].unique())

73917

# Anotações

* um franqueado pode ter mais de uma loja
* Revendedor fazem as vendas de porta em porta

-
* 504 franqueados
* 892 Lojas
* 73917 Revendedores

## Analise da categoria

In [10]:
categoria = base.groupby(['Produto_Categoria', 'Ciclo']).\
    agg({'Receita_Bruta':'sum', 'Receita_Liquida':'sum', \
        'Desconto':'sum', 'Pedido_Valor_Volume':'sum', 'z_Numero_de_Pedidos':'max'}).round(3).reset_index().\
    rename(columns={"Pedido_Valor_Volume": "Qtde_itens", 'z_Numero_de_Pedidos': 'Numero_de_Pedidos'})

fig = px.bar(categoria, x="Produto_Categoria", y="Receita_Bruta", \
    text="Ciclo", color="Ciclo", hover_data=['Ciclo', 'Qtde_itens', 'Numero_de_Pedidos'], template="seaborn")

fig.update_layout(
    margin=dict(l=20, r=50, t=50, b=20),
    title = 'Receita Bruta - Categorias',
    width=700, 
    height=600,
    uniformtext_minsize=12, 
    uniformtext_mode='hide',
)
fig.show()

In [11]:
fig = px.bar(categoria, x="Produto_Categoria", y="Desconto", \
    text="Desconto", color="Ciclo", hover_data=['Ciclo', 'Qtde_itens', 'Numero_de_Pedidos'], template="seaborn")

fig.update_layout(
    margin=dict(l=20, r=50, t=50, b=20),
    title = 'Receita Descontos - Categorias',
    width=700, 
    height=600,
    uniformtext_minsize=10, 
    uniformtext_mode='hide',
)
fig.show()

In [12]:
base.columns

Index(['Ciclo_Ciclo', 'Revendedor', 'Franqueado', 'Loja', 'Produto_Categoria',
       'Produto_Codigo', 'Produto_Descricao', 'Produto_Linha',
       'Pedido_Valor_Volume', 'Receita_Bruta', 'Receita_Liquida',
       'z_Numero_de_Pedidos', 'Desconto', 'Ciclo'],
      dtype='object')

In [13]:
revendedores = base.groupby(['Revendedor', 'Ciclo', 'Franqueado', 'Loja']).\
    agg({'Receita_Bruta':'sum', 'Receita_Liquida':'sum', \
        'Desconto':'sum', 'Pedido_Valor_Volume':'sum', 'z_Numero_de_Pedidos':'nunique'}).round(3).reset_index().\
    rename(columns={"Pedido_Valor_Volume": "Qtde_itens", 'z_Numero_de_Pedidos': 'Numero_de_Pedidos'}).sort_values(by='Receita_Bruta', ascending=False)

In [14]:
revendedores['Receita_Bruta'].sum()#.head(20).sort_values(by='Numero_de_Pedidos', ascending=False)

8163363.859999999

# Desempenho dos revendedores baseado na receita liquida

In [15]:
#Fevereiro
revendedores[revendedores['Ciclo'] == 'Fev-2020'].sort_values(by='Receita_Liquida', ascending=False).head(10)

,Revendedor,Ciclo,Franqueado,Loja,Receita_Bruta,Receita_Liquida,Desconto,Qtde_itens,Numero_de_Pedidos
19300,4080295,Fev-2020,10005,13909,1592.0,1592.00,0.00,8,1
8494,1243237,Fev-2020,10039,13347,1280.3,1088.36,191.94,27,2
34630,6999063,Fev-2020,10383,19975,1255.6,1067.10,188.50,44,2
25063,5596406,Fev-2020,10126,17292,796.0,796.00,0.00,4,1
31866,6692822,Fev-2020,10164,13480,796.0,796.00,0.00,4,1
8363,1220280,Fev-2020,10439,13437,897.0,762.60,134.40,30,1
20067,4252834,Fev-2020,13031,13125,756.4,756.40,0.00,9,2
13464,2538843,Fev-2020,13031,13125,614.4,614.40,0.00,12,1
22014,4667496,Fev-2020,10351,11105,597.0,597.00,0.00,3,1
1192,111563,Fev-2020,10522,14750,597.0,597.00,0.00,3,1


In [16]:
#Março
revendedores[revendedores['Ciclo'] == 'Mar-2020'].sort_values(by='Receita_Liquida', ascending=False).head(10)

,Revendedor,Ciclo,Franqueado,Loja,Receita_Bruta,Receita_Liquida,Desconto,Qtde_itens,Numero_de_Pedidos
9363,1406340,Mar-2020,10825,13658,796.0,796.00,0.00,4,1
30496,6543490,Mar-2020,13031,20890,796.0,796.00,0.00,4,1
30840,6580794,Mar-2020,11143,11671,796.0,796.00,0.00,4,1
68863,7771196,Mar-2020,13031,20890,746.0,746.00,0.00,4,1
17533,3655056,Mar-2020,10074,14363,846.6,719.44,127.16,34,1
19527,4132723,Mar-2020,10133,18228,788.5,670.30,118.20,15,1
2880,310534,Mar-2020,10282,20090,747.0,634.80,112.20,30,1
13730,2612533,Mar-2020,10857,20122,597.0,597.00,0.00,3,1
12637,2234282,Mar-2020,10066,14044,597.0,597.00,0.00,3,2
34129,6941062,Mar-2020,10009,3879,597.0,597.00,0.00,3,1


# Desempenho dos franqueados

In [58]:
franqueado = revendedores.groupby(['Franqueado', 'Ciclo']).agg({'Receita_Bruta':'sum', \
    'Receita_Liquida':'sum', 'Desconto':'sum', 'Loja':'nunique', 'Revendedor':'nunique', 'Qtde_itens':'sum', 'Numero_de_Pedidos':'sum'}).\
        round(3).reset_index().rename(columns={"Loja": "Qtde_Lojas", "Revendedor": "Qtde_Rev", "Numero_de_Pedidos":"N_pedidos"}).sort_values(by='Receita_Bruta', ascending=False)

franqueado['Liq_Med_p_Item'] = franqueado.apply(lambda x: round((x['Receita_Liquida'] / x['Qtde_itens']),2), axis=1)
franqueado['Desc_Med_p_Item'] = franqueado.apply(lambda x: round((x['Desconto'] / x['Qtde_itens']),2), axis=1)
franqueado['Desc_Med_p_Loja'] = franqueado.apply(lambda x: round((x['Desconto'] / x['Qtde_Lojas']),2), axis=1)
franqueado['Receita_Liq_p_Loja'] = franqueado.apply(lambda x: round((x['Receita_Liquida'] / x['Qtde_Lojas']),2), axis=1)
franqueado['Receita_Liq_p_Rev'] = franqueado.apply(lambda x: round((x['Receita_Liquida'] / x['Qtde_Rev']),2), axis=1)

In [59]:
#Maiores receitas liquidas - Franqueados
franqueado.head(10)

,Franqueado,Ciclo,Receita_Bruta,Receita_Liquida,Desconto,Qtde_Lojas,Qtde_Rev,Qtde_itens,N_pedidos,Liq_Med_p_Item,Desc_Med_p_Item,Desc_Med_p_Loja,Receita_Liq_p_Loja,Receita_Liq_p_Rev
981,13031,Fev-2020,636000.95,619289.66,16711.29,25,6266,8073,6284,76.71,2.07,668.45,24771.59,98.83
982,13031,Mar-2020,596118.34,580552.00,15566.34,25,6506,8433,6534,68.84,1.85,622.65,23222.08,89.23
323,10269,Fev-2020,117757.25,116600.11,1157.14,17,872,958,872,121.71,1.21,68.07,6858.83,133.72
391,10351,Fev-2020,94463.02,93382.36,1080.66,2,761,859,761,108.71,1.26,540.33,46691.18,122.71
392,10351,Mar-2020,67896.12,66622.18,1273.94,2,622,726,623,91.77,1.75,636.97,33311.09,107.11
324,10269,Mar-2020,66188.63,64961.33,1227.30,17,584,657,585,98.88,1.87,72.19,3821.25,111.24
991,19825,Fev-2020,61445.30,60599.26,846.04,2,511,578,511,104.84,1.46,423.02,30299.63,118.59
992,19825,Mar-2020,55112.25,54236.61,875.64,2,484,615,485,88.19,1.42,437.82,27118.31,112.06
965,11334,Fev-2020,52635.48,52475.60,159.88,9,355,359,355,146.17,0.45,17.76,5830.62,147.82
213,10169,Mar-2020,48156.42,47330.29,826.13,2,420,571,422,82.89,1.45,413.06,23665.15,112.69


In [60]:
#Maiores Receitas liquidas por Loja
franqueado.sort_values(by='Receita_Liq_p_Loja', ascending=False).head(10)

,Franqueado,Ciclo,Receita_Bruta,Receita_Liquida,Desconto,Qtde_Lojas,Qtde_Rev,Qtde_itens,N_pedidos,Liq_Med_p_Item,Desc_Med_p_Item,Desc_Med_p_Loja,Receita_Liq_p_Loja,Receita_Liq_p_Rev
391,10351,Fev-2020,94463.02,93382.36,1080.66,2,761,859,761,108.71,1.26,540.33,46691.18,122.71
184,10153,Fev-2020,36464.40,36173.24,291.16,1,309,328,309,110.28,0.89,291.16,36173.24,117.07
10,10009,Fev-2020,34394.61,33975.31,419.30,1,272,312,272,108.90,1.34,419.30,33975.31,124.91
392,10351,Mar-2020,67896.12,66622.18,1273.94,2,622,726,623,91.77,1.75,636.97,33311.09,107.11
991,19825,Fev-2020,61445.30,60599.26,846.04,2,511,578,511,104.84,1.46,423.02,30299.63,118.59
202,10164,Fev-2020,30451.40,30036.08,415.32,1,266,311,266,96.58,1.34,415.32,30036.08,112.92
203,10164,Mar-2020,29484.69,29132.81,351.88,1,240,283,240,102.94,1.24,351.88,29132.81,121.39
11,10009,Mar-2020,28316.50,27951.28,365.22,1,274,363,274,77.00,1.01,365.22,27951.28,102.01
992,19825,Mar-2020,55112.25,54236.61,875.64,2,484,615,485,88.19,1.42,437.82,27118.31,112.06
981,13031,Fev-2020,636000.95,619289.66,16711.29,25,6266,8073,6284,76.71,2.07,668.45,24771.59,98.83


In [61]:
#Maiores Receitas liquidas por Rev
franqueado.sort_values(by='Receita_Liq_p_Rev', ascending=False).head(10)

,Franqueado,Ciclo,Receita_Bruta,Receita_Liquida,Desconto,Qtde_Lojas,Qtde_Rev,Qtde_itens,N_pedidos,Liq_Med_p_Item,Desc_Med_p_Item,Desc_Med_p_Loja,Receita_Liq_p_Loja,Receita_Liq_p_Rev
242,10195,Fev-2020,845.00,845.00,0.00,1,4,5,4,169.00,0.00,0.00,845.00,211.25
338,10280,Mar-2020,6394.90,6383.82,11.08,1,31,37,31,172.54,0.30,11.08,6383.82,205.93
787,10695,Mar-2020,547.00,547.00,0.00,1,3,3,3,182.33,0.00,0.00,547.00,182.33
421,10373,Fev-2020,547.00,547.00,0.00,1,3,3,3,182.33,0.00,0.00,547.00,182.33
930,10825,Mar-2020,1964.05,1946.09,17.96,1,11,16,11,121.63,1.12,17.96,1946.09,176.92
768,10682,Mar-2020,348.00,348.00,0.00,1,2,2,2,174.00,0.00,0.00,348.00,174.00
52,10039,Fev-2020,2823.70,2608.88,214.82,1,15,42,16,62.12,5.11,214.82,2608.88,173.93
305,10253,Mar-2020,19708.30,19631.38,76.92,1,114,123,114,159.60,0.63,76.92,19631.38,172.21
342,10283,Mar-2020,13542.10,13471.86,70.24,1,79,89,79,151.37,0.79,70.24,13471.86,170.53
243,10195,Mar-2020,845.00,845.00,0.00,1,5,5,5,169.00,0.00,0.00,845.00,169.00


In [62]:
#Maiores Desconto liquido por item
franqueado.sort_values(by='Desc_Med_p_Item', ascending=False).head(10)

,Franqueado,Ciclo,Receita_Bruta,Receita_Liquida,Desconto,Qtde_Lojas,Qtde_Rev,Qtde_itens,N_pedidos,Liq_Med_p_Item,Desc_Med_p_Item,Desc_Med_p_Loja,Receita_Liq_p_Loja,Receita_Liq_p_Rev
872,10773,Fev-2020,149.8,127.34,22.46,1,2,2,2,63.67,11.23,22.46,127.34,63.67
498,10428,Mar-2020,375.4,319.12,56.28,1,6,6,6,53.19,9.38,56.28,319.12,53.19
563,10480,Fev-2020,119.8,101.84,17.96,1,1,2,1,50.92,8.98,17.96,101.84,101.84
903,10802,Mar-2020,59.9,50.92,8.98,1,1,1,1,50.92,8.98,8.98,50.92,50.92
639,10546,Fev-2020,468.5,420.58,47.92,1,6,6,6,70.10,7.99,47.92,420.58,70.10
727,10633,Fev-2020,159.7,135.76,23.94,1,3,3,3,45.25,7.98,23.94,135.76,45.25
801,10706,Mar-2020,105.8,89.94,15.86,1,2,2,2,44.97,7.93,15.86,89.94,44.97
702,10604,Mar-2020,104.8,89.08,15.72,1,2,2,2,44.54,7.86,15.72,89.08,44.54
640,10546,Mar-2020,516.0,438.64,77.36,1,7,10,7,43.86,7.74,77.36,438.64,62.66
742,10651,Mar-2020,249.5,212.10,37.40,1,5,5,5,42.42,7.48,37.40,212.10,42.42


## Calculando o preço médio por  produto agrupado por ciclo

In [17]:
prec_med_fev = revendedores[revendedores['Ciclo']=='Fev-2020']['Receita_Bruta'].sum() /\
     revendedores[revendedores['Ciclo']=='Fev-2020']['Qtde_itens'].sum()
prec_med_fev

92.48508834376877

In [18]:
prec_med_mar = revendedores[revendedores['Ciclo']=='Mar-2020']['Receita_Bruta'].sum() /\
     revendedores[revendedores['Ciclo']=='Mar-2020']['Qtde_itens'].sum()
prec_med_mar

80.90076651538074

## Desembolso medio por revendedora

In [19]:
des_med_por_rev_fev = revendedores[revendedores['Ciclo']=='Fev-2020']['Receita_Bruta'].sum() /\
     len(revendedores[revendedores['Ciclo']=='Fev-2020']['Revendedor'].unique())
des_med_por_rev_fev

112.39731536978162

In [20]:
des_med_por_rev_mar = revendedores[revendedores['Ciclo']=='Mar-2020']['Receita_Bruta'].sum() /\
     len(revendedores[revendedores['Ciclo']=='Mar-2020']['Revendedor'].unique())
des_med_por_rev_mar

102.17688233108646

## Média de itens por boleto

In [21]:
itens_por_boleto_fev = revendedores[revendedores['Ciclo'] == 'Fev-2020']['Qtde_itens'].sum() /\
     revendedores[revendedores['Ciclo'] == 'Fev-2020']['Numero_de_Pedidos'].sum()
itens_por_boleto_fev

1.213341658861484

In [22]:
itens_por_boleto_mar = revendedores[revendedores['Ciclo'] == 'Mar-2020']['Qtde_itens'].sum() /\
     revendedores[revendedores['Ciclo'] == 'Mar-2020']['Numero_de_Pedidos'].sum()
itens_por_boleto_mar

1.259814172644731

In [23]:
base.columns

Index(['Ciclo_Ciclo', 'Revendedor', 'Franqueado', 'Loja', 'Produto_Categoria',
       'Produto_Codigo', 'Produto_Descricao', 'Produto_Linha',
       'Pedido_Valor_Volume', 'Receita_Bruta', 'Receita_Liquida',
       'z_Numero_de_Pedidos', 'Desconto', 'Ciclo'],
      dtype='object')

# Basket analisys - Algoritimo Apriori

## ------ Fevereiro - 2020 ----------

In [24]:
#criando um id para separar os boletos
base['id_basket'] = base.apply(lambda x: str(x['Revendedor']) + '-' + str(x['z_Numero_de_Pedidos']), axis=1)

basket_plus = base[base['Ciclo'] == 'Fev-2020'].groupby(['id_basket','Produto_Descricao'])['Pedido_Valor_Volume'].\
    sum().unstack().reset_index().fillna(0).set_index('id_basket')

#encode
basket_plus = basket_plus.applymap(lambda x: 0 if x<=0 else 1)

print(len(basket_plus))

#filtrando os que tem mais de um item
basket_plus = basket_plus[(basket_plus > 0).sum(axis=1) >= 2]

print(len(basket_plus))


38436
1424


In [25]:
#Aplicando o algoritmo Apriori
frequent_itens = apriori(basket_plus, min_support=0.03, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

In [26]:
frequent_itens['length'] = frequent_itens['itemsets'].apply(lambda x: len(x))
frequent_itens

,support,itemsets,length
0,0.319522,(BOT PERFUME LILLY 75ML - ED ESPECIAL),1
1,0.231039,(BOTI DEOS AEROSOL MALBEC 40ML),1
2,0.162921,(BOT BLOCO DE PEDIDOS VD),1
3,0.105337,(BOT SABONETE LÍQ LILLY 50ML),1
4,0.089888,(BOTI MASCARA PROT MALHA CINZA MESCLA M),1
5,0.089185,(CATALOGO CICLO PROXIMO SELL OUT),1
6,0.083567,(BOTI MASCARA PROT MALHA PRETA M),1
7,0.077247,(BOTI SPLASH MALBEC 100ML),1
8,0.074438,(BOTI NECESS ENVELOPE PU ROSA C/A),1
9,0.072331,(BOTI MASCARA PROT MALHA PRETA G),1


In [27]:
#criando as regras de associação
association_rules(frequent_itens, metric='lift', min_threshold=0.8).sort_values(by='lift', ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(BOTI NECESS ENVELOPE DOURADO PU C/A),(BOTI NECESS ENVELOPE PU ROSA C/A),0.057584,0.074438,0.030899,0.536585,7.208468,0.026612,1.997265
1,(BOTI NECESS ENVELOPE PU ROSA C/A),(BOTI NECESS ENVELOPE DOURADO PU C/A),0.074438,0.057584,0.030899,0.415094,7.208468,0.026612,1.611227
2,(BOTI MASCARA PROT MALHA CINZA MESCLA M),(BOTI MASCARA PROT MALHA PRETA M),0.089888,0.083567,0.053371,0.593750,7.105042,0.045859,2.255834
3,(BOTI MASCARA PROT MALHA PRETA M),(BOTI MASCARA PROT MALHA CINZA MESCLA M),0.083567,0.089888,0.053371,0.638655,7.105042,0.045859,2.518683
4,(BOTI MASCARA PROT MALHA CINZA MESCLA M),(BOTI MASCARA PROT MALHA PRETA G),0.089888,0.072331,0.035112,0.390625,5.400485,0.028611,1.522328
5,(BOTI MASCARA PROT MALHA PRETA G),(BOTI MASCARA PROT MALHA CINZA MESCLA M),0.072331,0.089888,0.035112,0.485437,5.400485,0.028611,1.768709
6,(BOT BLOCO DE PEDIDOS VD),(CATALOGO CICLO PROXIMO SELL OUT),0.162921,0.089185,0.035112,0.215517,2.416508,0.020582,1.161038
7,(CATALOGO CICLO PROXIMO SELL OUT),(BOT BLOCO DE PEDIDOS VD),0.089185,0.162921,0.035112,0.393701,2.416508,0.020582,1.380636
8,(BOT PERFUME LILLY 75ML - ED ESPECIAL),(BOT BLOCO DE PEDIDOS VD),0.319522,0.162921,0.046348,0.145055,0.890337,-0.005709,0.979102
9,(BOT BLOCO DE PEDIDOS VD),(BOT PERFUME LILLY 75ML - ED ESPECIAL),0.162921,0.319522,0.046348,0.284483,0.890337,-0.005709,0.951029


## ------ Março - 2020 ----------

In [28]:
basket_plus_mar = base[base['Ciclo'] == 'Mar-2020'].groupby(['id_basket','Produto_Descricao'])['Pedido_Valor_Volume'].\
    sum().unstack().reset_index().fillna(0).set_index('id_basket')

#encode
basket_plus_mar = basket_plus_mar.applymap(lambda x: 0 if x<=0 else 1)

print(len(basket_plus_mar))

#filtrando os que tem mais de um item
basket_plus_mar = basket_plus_mar[(basket_plus_mar > 0).sum(axis=1) >= 2]

print(len(basket_plus_mar))


37777
2046


In [29]:
#Aplicando o algoritmo Apriori
frequent_itens_mar = apriori(basket_plus_mar, min_support=0.03, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

In [30]:
frequent_itens_mar['length'] = frequent_itens_mar['itemsets'].apply(lambda x: len(x))
frequent_itens_mar

,support,itemsets,length
0,0.246334,(BOT PERFUME LILLY 75ML - ED ESPECIAL),1
1,0.194526,(CATALOGO CICLO PROXIMO SELL OUT),1
2,0.180352,(BOTI NECESS ENVELOPE PU ROSA C/A),1
3,0.165200,(BOT BLOCO DE PEDIDOS VD),1
4,0.155425,(BOTI NECESS ENVELOPE DOURADO PU C/A),1
5,0.152493,(BOTI MASCARA PROT MALHA PRETA M),1
6,0.138807,(BOTI DEOS AEROSOL MALBEC 40ML),1
7,0.111437,(BOTI MASCARA PROT MALHA CINZA MESCLA M),1
8,0.107527,"(BOTI NECESS ENVELOPE DOURADO PU C/A, BOTI NEC...",2
9,0.086999,(BOTI FITA OLFATIVA VD),1


In [31]:
#criando as regras de associação
association_rules(frequent_itens_mar, metric='lift', min_threshold=0.8).sort_values(by='lift', ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(BOTI MASCARA PROT MALHA PRETA M),(BOTI MASCARA PROT MALHA CINZA MESCLA M),0.152493,0.111437,0.081623,0.535256,4.803222,0.064629,1.911943
1,(BOTI MASCARA PROT MALHA CINZA MESCLA M),(BOTI MASCARA PROT MALHA PRETA M),0.111437,0.152493,0.081623,0.732456,4.803222,0.064629,3.167732
2,(CATALOGO CICLO ATUAL SELL OUT),(CATALOGO CICLO PROXIMO SELL OUT),0.081623,0.194526,0.068426,0.838323,4.309572,0.052548,4.982006
3,(CATALOGO CICLO PROXIMO SELL OUT),(CATALOGO CICLO ATUAL SELL OUT),0.194526,0.081623,0.068426,0.351759,4.309572,0.052548,1.416722
4,(BOTI NECESS ENVELOPE PU ROSA C/A),(BOTI NECESS ENVELOPE DOURADO PU C/A),0.180352,0.155425,0.107527,0.596206,3.835967,0.079496,2.091598
5,(BOTI NECESS ENVELOPE DOURADO PU C/A),(BOTI NECESS ENVELOPE PU ROSA C/A),0.155425,0.180352,0.107527,0.691824,3.835967,0.079496,2.659674
6,(BOTI FITA OLFATIVA VD),(BOT BLOCO DE PEDIDOS VD),0.086999,0.165200,0.043011,0.494382,2.992620,0.028638,1.651048
7,(BOT BLOCO DE PEDIDOS VD),(BOTI FITA OLFATIVA VD),0.165200,0.086999,0.043011,0.260355,2.992620,0.028638,1.234377
8,(BOT BLOCO DE PEDIDOS VD),(CATALOGO CICLO PROXIMO SELL OUT),0.165200,0.194526,0.055230,0.334320,1.718638,0.023094,1.210001
9,(CATALOGO CICLO PROXIMO SELL OUT),(BOT BLOCO DE PEDIDOS VD),0.194526,0.165200,0.055230,0.283920,1.718638,0.023094,1.165790
